In [84]:
import requests 
import pandas as pd
import numpy as np
import sys; sys.path
import datetime
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random

from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train, read_result
from bitcoin_deep_learning.cross_val import cross_val
api = ApiCall()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
#from google.colab import drive
#drive.mount('/content/drive')

In [74]:
import requests 
import pandas as pd
import numpy as np
import sys; sys.path
import datetime
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import random

from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train
from bitcoin_deep_learning.cross_val import cross_val

%load_ext autoreload
%autoreload 2
from bitcoin_deep_learning.call_api import ApiCall
from bitcoin_deep_learning.model import LinearRegressionBaselineModel, RnnDlModel
from bitcoin_deep_learning.trainer import train
from bitcoin_deep_learning.cross_val import cross_val,fold_indexes,sequence_indexes


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
train_df = ApiCall().read_local()
print((9014.026207-8222.753649)/9014.026207)
print((8799.121911-8677.910527)/8799.121911)
train_df.head(20)


0.08778236714971208
0.013775395457184293


,date,[AVG]_[NH]_mean_hash_rate,[+]_[NH]_Days_Till_Halving,[//]_[AV]_Stock-to-Flow_Ratio,[+]_[NH]_Circulating_Supply,[+]_[NH]_Issuance,[+]_[NH]_Number_of_Active_Addresses,[+]_[NH]_Mean_Block_Interval,[+]_[NH]_Number_of_Transactions,[+]_[NH]_Number_of_Addresses_with_a_Non-Zero_Balance,...,[%]_Bitcoin_growth_rate_on_Horizon=7,fear_greed_value,[+]_[NH]_Number_of_Addresses_with_Balance_0.01 - 0.1,[+]_[NH]_Number_of_Addresses_with_Balance_0.1 - 1,[+]_[NH]_Number_of_Addresses_with_Balance_1 - 10,[+]_[NH]_Number_of_Addresses_with_Balance_10 - 100,[+]_[NH]_Number_of_Addresses_with_Balance_100 - 1k,[+]_[NH]_Number_of_Addresses_with_Balance_1k - 10k,[+]_[T]_Bitcoin_Price,[%]_Bitcoin_growth_rate_on_Horizon=7.1
0,2018-02-01,2.054109e+19,830,5642.903220,16839700.0,2000.0,842398,377,257504,27026398,...,-0.096230,30,4226306,1722331,544769,132314,15779,1529,9014.026207,-0.096230
1,2018-02-02,2.170824e+19,829,5641.695528,16841800.0,2100.0,854253,385,235750,26955617,...,-0.013968,15,4198274,1704275,543489,132213,15815,1521,8799.121911,-0.013968
2,2018-02-03,2.073919e+19,828,5641.015128,16843775.0,1975.0,714655,384,194733,26886977,...,-0.078603,40,4173275,1692828,542447,132021,15812,1523,9208.459919,-0.078603
3,2018-02-04,2.280248e+19,827,5632.191528,16846000.0,2225.0,720904,349,173509,26753036,...,-0.012779,24,4103680,1674480,540698,132009,15830,1532,8176.460616,-0.012779
4,2018-02-05,2.411984e+19,826,5626.186626,16848312.5,2312.5,903239,338,216178,26562089,...,0.224166,11,4036425,1668996,541030,131987,15813,1519,6891.934414,0.224166
5,2018-02-06,2.199977e+19,825,5613.582892,16850437.5,2125.0,999110,358,243950,26426352,...,0.094468,8,4034268,1663664,537892,131953,15822,1525,7714.392057,0.094468
6,2018-02-07,2.004877e+19,824,5619.041374,16852200.0,1762.5,814500,453,213578,26340551,...,0.201176,36,4024022,1659967,538766,131816,15782,1521,7575.611197,0.201176
7,2018-02-08,1.876627e+19,823,5626.039833,16853837.5,1637.5,746341,435,173158,26217915,...,0.182946,30,4010185,1662072,539783,131822,15806,1527,8222.753649,0.182946
8,2018-02-09,1.919364e+19,822,5639.520131,16855525.0,1687.5,775346,483,177725,26145417,...,0.152258,44,4006306,1666049,540436,131315,15745,1529,8677.910527,0.152258
9,2018-02-10,2.043647e+19,821,5643.008314,16857312.5,1787.5,741833,426,181640,26044328,...,0.230983,54,3992860,1670898,539736,131438,15733,1533,8537.393208,0.230983


In [83]:
model = LinearRegressionBaselineModel2()
folf_score,score = train(model,train_df)
print(fold_score,score)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  5.94it/s]

Training done
[25.11065, 40.48095, 72.67151, 43.94785, 11.77665, 34.83829, 57.82235, 63.08738, 66.34564, 29.20328, 20.43035, 22.46888, 60.14261, 842.76034, 2000.52293, 1453.94488, 56.86854, 18.818, 24.60637, 24.41708, 57363387598.00065, 31251929429499.957, 37608356950071.5, 3037.49666, 47.81047, 121.95399, 243.33564, 264.62119, 77.54508, 40.13575, 21.42994, 16.17604] 8.748384595108662e+24


In [97]:
model = LinearRegressionBaselineModel(alpha=50,l1_ratio=0.01)
folf_score,score = train(model,train_df)
print(fold_score,score)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  5.92it/s]

Training done
[25.11065, 40.48095, 72.67151, 43.94785, 11.77665, 34.83829, 57.82235, 63.08738, 66.34564, 29.20328, 20.43035, 22.46888, 60.14261, 842.76034, 2000.52293, 1453.94488, 56.86854, 18.818, 24.60637, 24.41708, 57363387598.00065, 31251929429499.957, 37608356950071.5, 3037.49666, 47.81047, 121.95399, 243.33564, 264.62119, 77.54508, 40.13575, 21.42994, 16.17604] 0.01254


In [80]:
for c in range counter() :
    alpha = random.random()

array([-3.74030919e-01,  3.71852553e+00, -6.15565923e-01,  4.34454746e+00,
        5.04449729e-01, -1.87755456e-02,  8.94154082e-02, -1.03440257e-03,
        4.53927321e-01, -7.18579181e-02, -4.24942756e-01,  1.29669038e-01,
        2.58950736e-02, -1.44874019e-01,  6.71343146e-02, -4.77944275e-02,
        4.58262987e-02, -1.18035888e-01, -1.28765215e+00, -9.80012731e-01,
       -3.16867534e-01, -2.31456153e-01,  3.96501869e-02, -3.87178009e-01,
       -1.34610973e-01, -3.35430246e-01,  2.82905460e-01,  2.64920297e-02,
        1.95285481e-01,  1.70184510e-01,  1.57998486e-01,  8.83474862e-01])

In [94]:
train(model,train_df)
read_result().sort_values(by="date")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:05<00:00,  5.91it/s]

Training done


,name,fold_score,mean_score,min_score,max_score,hyperparams,date
0,ClassicLinearReg,"[0.07596, 0.15033, 0.40177, 0.76549, 0.14513, ...",8.748385e+24,0.05,1.856245e+26,None,08-03 09:39:35
1,ClassicLinearReg,"[0.07596, 0.15033, 0.40177, 0.76549, 0.14513, ...",8.748385e+24,0.05,1.856245e+26,None,08-03 09:50:46


In [98]:
X_train_list,Y_train_listget_cross_XY()

32


([array([[[ 2.05410932e+19,  8.30000000e+02,  5.64290322e+03, ...,
            1.52900000e+03,  9.01402621e+03, -9.62296320e-02],
          [ 2.17082412e+19,  8.29000000e+02,  5.64169553e+03, ...,
            1.52100000e+03,  8.79912191e+03, -1.39678074e-02],
          [ 2.07391938e+19,  8.28000000e+02,  5.64101513e+03, ...,
            1.52300000e+03,  9.20845992e+03, -7.86032334e-02],
          ...,
          [ 3.16612395e+19,  7.43000000e+02,  5.72810731e+03, ...,
            1.54200000e+03,  9.41509000e+03,  2.40132936e-02],
          [ 3.11301327e+19,  7.42000000e+02,  5.72907144e+03, ...,
            1.53900000e+03,  9.24719000e+03,  1.30898713e-02],
          [ 3.14666852e+19,  7.41000000e+02,  5.72871501e+03, ...,
            1.53600000e+03,  9.10553000e+03,  1.55917728e-02]],
  
         [[ 2.17082412e+19,  8.29000000e+02,  5.64169553e+03, ...,
            1.52100000e+03,  8.79912191e+03, -1.39678074e-02],
          [ 2.07391938e+19,  8.28000000e+02,  5.64101513e+03, ...,
    